In [4]:
import pandas as pd
import json

# JSONを読み込む
## シンプルな構造のJSON
[link: JSONファイルをフラットなpandasデータフレームに変換](https://qiita.com/kawada2017/items/937b6bd5ea81ea7974ad "JSONファイルをフラットなpandasデータフレームに変換")

In [2]:
# sample1
json_file='sample1.json'
df = pd.read_json(json_file)
df

,id,UP_TIME,POWER,TEMP,ERR_CD
0,1000,0,948,250,NaN
1,1000,1,945,251,1.0


## ネスト構造

In [3]:
# sample2をまずはそのまま読み込む
json_file='sample2.json'
df = pd.read_json(json_file)
df

,id,sensor,code
0,"{'M_CD': 1000, 'UP_TIME': 0}","{'POWER': 948, 'TEMP': 250}",NaN
1,"{'M_CD': 1000, 'UP_TIME': 1}","{'POWER': 945, 'TEMP': 251}",{'ERR_CD': 1}


- 読み込むことはできるが入れ子構造がうまく解析されていない
## json.loadで配列に読み出す

In [5]:
with open(json_file, encoding='utf-8') as f:
    nest_json_data = json.load(f)

# pandasでノーマライズ
df = pd.json_normalize(nest_json_data)
df

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.ERR_CD
0,1000,0,948,250,NaN
1,1000,1,945,251,1.0


## ネスト構造かつリストを含む場合
- まずはノーマライズしてみる

In [6]:
json_file = 'sample3.json'

with open(json_file, encoding='utf-8') as f:
    nest_json_data = json.load(f)

# pandasでノーマライズ
df = pd.json_normalize(nest_json_data)
df

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code
0,1000,0,948,250,NaN
1,1000,1,945,251,"[{'ERR_CD': 1, 'MESSAGE': 'part1'}]"
2,1000,2,943,255,"[{'ERR_CD': 2, 'MESSAGE': 'part2'}, {'ERR_CD':..."


## フラット化しPandasで読み込む関数

In [8]:
# フラット化
def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        # 列名の生成
        new_key = parent_key + sep + k if parent_key else k
        # 辞書型項目のフラット化
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        # リスト項目のフラット化
        elif isinstance(v, list):
            new_key_tmp = new_key
            for i, elm in enumerate(v):
                new_key = new_key_tmp + sep + str(i)
                # リストの中の辞書
                if isinstance(elm, dict):
                    items.extend(flatten(elm, new_key, sep=sep).items())
                # 単なるリスト
                else:
                    items.append((new_key, elm))
        # 値追加
        else:
            items.append((new_key, v))
    return dict(items)


def flattenJsonFile(jsonfile, rowsroot, sep='.'):
    """
    JSONファイルを読み込み2次元のpandas DataFrameに変換する

    Parameters
    ----------
    jsonfile : string
        JSONファイルパス
    rowsroot : string
        フラット化するルートエレメント名。トップからでいい場合は空文字を入力する
    sep : string
        ノーマライズされていないエレメントを区切る文字

    Returns
    -------
    df : pandas.DataFrame
        フラット化されたpandas DataFrame
    """
    # JSONファイルを読込
    with open(jsonfile, encoding='utf-8') as f:
        d = json.load(f)

    # df化したい辞書リストのルート項目を指定
    if rowsroot != '':
        d = d[rowsroot]

    # フラット化
    dlist = []
    for di in d:
        dlist.append(flatten(di, sep=sep))

    # フラット化された辞書をpandasデータフレームに変換
    return pd.DataFrame.from_dict(dlist)


In [11]:
flattenJsonFile('sample3.json','')

,id.M_CD,id.UP_TIME,sensor.POWER,sensor.TEMP,code.0.ERR_CD,code.0.MESSAGE,code.1.ERR_CD,code.1.MESSAGE
0,1000,0,948,250,NaN,NaN,NaN,NaN
1,1000,1,945,251,1.0,part1,NaN,NaN
2,1000,2,943,255,2.0,part2,3.0,part3


In [ ]:
df_automate = flattenJsonFile('power_automate.json','')
df_automate

## resultsのJSONのみ抽出する

In [ ]:
# まずはシリアライズ
json_file = 'power_automate.json'

with open(json_file, encoding='utf-8') as f:
    nest_json_data = json.load(f)

# pandasでノーマライズ
df = pd.json_normalize(nest_json_data)
df.to_csv('power_automate.csv', index=False)


## Power AutomateのJSONを解析したい場合はネストされた項目から必要項目のみ抜き出した方が扱いやすい
- flattenで1行xx列のデータにすると、ユーザーID毎に行を作成できないため